## Setup

In [5]:
import os
import pandas as pd

data_dir = "Data/"

#data_dir = "../Data/"

# Will fix the copy warning but for now, ignore them 
pd.set_option('mode.chained_assignment', None)

#### ICD-9 Mapping

In [6]:
icd9_diag = pd.read_excel(data_dir+"ICD-9 Diag Descriptions.xlsx")

icd9_diag_dict = icd9_diag.set_index("DIAGNOSIS CODE")["LONG DESCRIPTION"].to_dict()

icd9_proc = pd.read_excel(data_dir+"ICD-9 Proc Descriptions.xls")

icd9_proc["PROCEDURE CODE"] = icd9_proc["PROCEDURE CODE"].astype(float)

icd9_proc_dict = icd9_proc.set_index("PROCEDURE CODE")["LONG DESCRIPTION"].to_dict()

In [7]:
icd9_diag_dict

{'0010': 'Cholera due to vibrio cholerae',
 '0011': 'Cholera due to vibrio cholerae el tor',
 '0019': 'Cholera, unspecified',
 '0020': 'Typhoid fever',
 '0021': 'Paratyphoid fever A',
 '0022': 'Paratyphoid fever B',
 '0023': 'Paratyphoid fever C',
 '0029': 'Paratyphoid fever, unspecified',
 '0030': 'Salmonella gastroenteritis',
 '0031': 'Salmonella septicemia',
 '00320': 'Localized salmonella infection, unspecified',
 '00321': 'Salmonella meningitis',
 '00322': 'Salmonella pneumonia',
 '00323': 'Salmonella arthritis',
 '00324': 'Salmonella osteomyelitis',
 '00329': 'Other localized salmonella infections',
 '0038': 'Other specified salmonella infections',
 '0039': 'Salmonella infection, unspecified',
 '0040': 'Shigella dysenteriae',
 '0041': 'Shigella flexneri',
 '0042': 'Shigella boydii',
 '0043': 'Shigella sonnei',
 '0048': 'Other specified shigella infections',
 '0049': 'Shigellosis, unspecified',
 '0050': 'Staphylococcal food poisoning',
 '0051': 'Botulism food poisoning',
 '0052': 

In [8]:
icd9_proc_dict

{1.0: 'Therapeutic ultrasound of vessels of head and neck',
 2.0: 'Therapeutic ultrasound of heart',
 3.0: 'Therapeutic ultrasound of peripheral vascular vessels',
 9.0: 'Other therapeutic ultrasound',
 10.0: 'Implantation of chemotherapeutic agent',
 11.0: 'Infusion of drotrecogin alfa (activated)',
 12.0: 'Administration of inhaled nitric oxide',
 13.0: 'Injection or infusion of nesiritide',
 14.0: 'Injection or infusion of oxazolidinone class of antibiotics',
 15.0: 'High-dose infusion interleukin-2 [IL-2]',
 16.0: 'Excision of lesion of skull',
 17.0: 'Infusion of vasopressor agent',
 18.0: 'Infusion of immunosuppressive antibody therapy',
 19.0: 'Disruption of blood brain barrier via infusion [BBBD]',
 21.0: 'Intravascular imaging of extracranial cerebral vessels',
 22.0: 'Ventriculostomy',
 23.0: 'Intravascular imaging of peripheral vessels',
 24.0: 'Intravascular imaging of coronary vessels',
 25.0: 'Intravascular imaging of renal vessels',
 28.0: 'Intravascular imaging, other s

#### MS-DRG Codes Map

In [9]:
ms_drgs = pd.read_csv(data_dir+"ms-drgs.csv")
ms_drgs["Code"] = ms_drgs["Code"].astype(str)

ms_drgs_dict= ms_drgs.set_index("Code")["Description"].to_dict()

In [10]:
ms_drgs_dict

{'1': 'Heart Transplant or Implant of Heart Assist System with MCC',
 '2': 'Heart Transplant or Implant of Heart Assist System without MCC',
 '3': 'ECMO or Tracheostomy with MV >96 Hours or Principal Diagnosis Except Face Mouth and Neck with Major O.R. Procedures',
 '4': 'Tracheostomy with MV >96 Hours or Principal Diagnosis Except Face Mouth and Neck without Major O.R. Procedures',
 '5': 'Liver Transplant with MCC or Intestinal Transplant',
 '6': 'Liver Transplant without MCC',
 '7': 'Lung Transplant',
 '8': 'Simultaneous Pancreas and Kidney Transplant',
 '10': 'Pancreas Transplant',
 '11': 'Tracheostomy for Face Mouth and Neck Diagnoses or Laryngectomy with MCC',
 '12': 'Tracheostomy for Face Mouth and Neck Diagnoses or Laryngectomy with CC',
 '13': 'Tracheostomy for Face Mouth and Neck Diagnoses or Laryngectomy without CC/MCC',
 '14': 'Allogeneic Bone Marrow Transplant',
 '16': 'Autologous Bone Marrow Transplant with CC/MCC',
 '17': 'Autologous Bone Marrow Transplant without CC/MCC'

In [ ]:
# TODO: Add Mapping for CCS Codes REF: 

In [ ]:
## TODO: Mapping for CC codes

## Cleaning

### Data Dictionary



| #  | Column Name                         | Description                                      | Notes |
|----|--------------------------------------|--------------------------------------------------|-------|
| 1  | DESYNPUF_ID                         | Beneficiary Code                                |       |
| 2  | CLM_ID                               | Claim ID                                        |       |
| 3  | SEGMENT                              | Claim Line Segment                              |       |
| 4  | CLM_FROM_DT                          | Claims start date                               |       |
| 5  | CLM_THRU_DT                          | Claims end date                                 |       |
| 6  | PRVDR_NUM                            | Provider Institution                            |       |
| 7  | CLM_PMT_AMT                          | Claim Payment Amount                            |       |
| 8  | NCH_PRMRY_PYR_CLM_PD_AMT             | NCH Primary Payer Claim Paid Amount            |       |
| 9  | AT_PHYSN_NPI                         | Attending Physician – National Provider Identifier Number |       |
| 10 | OP_PHYSN_NPI                         | Operating Physician – National Provider Identifier Number |       |
| 11 | OT_PHYSN_NPI                         | Other Physician – National Provider Identifier Number |       |
| 12 | CLM_ADMSN_DT                         | Inpatient admission date                        |       |
| 13 | ADMTNG_ICD9_DGNS_CD                  | Claim Admitting Diagnosis Code                 | Irrelevant for us  |
| 14 | CLM_PASS_THRU_PER_DIEM_AMT           | Claim Pass Thru Per Diem Amount                |       |
| 15 | NCH_BENE_IP_DDCTBL_AMT               | NCH Beneficiary Inpatient Deductible Amount    |       |
| 16 | NCH_BENE_PTA_COINSRNC_LBLTY_AM       | NCH Beneficiary Part A Coinsurance Liability Amount |       |
| 17 | NCH_BENE_BLOOD_DDCTBL_LBLTY_AM       | NCH Beneficiary Blood Deductible Liability Amount |       |
| 18 | CLM_UTLZTN_DAY_CNT                   | Claim Utilization Day Count                    |       |
| 19 | NCH_BENE_DSCHRG_DT                   | Inpatient discharged date                      |       |
| 20 | CLM_DRG_CD                           | Claim Diagnosis Related Group Code             |       |
| 21-30 | ICD9_DGNS_CD_1 – ICD9_DGNS_CD_10  | Claim Diagnosis Code 1 – Claim Diagnosis Code 10 |       |
| 31-36 | ICD9_PRCDR_CD_1 – ICD9_PRCDR_CD_6 | Claim Procedure Code 1 – Claim Procedure Code 6 |       |
| 37-81 | HCPCS_CD_1 – HCPCS_CD_45          | Claim HCPCS Code 1 – Claim HCPCS Code 45       | Removed later as all columns are null       |


In [11]:
df = pd.read_csv(data_dir+"inpatient.csv")

### Format Dates to be DateTime for later calculations

In [12]:
df['NCH_BENE_DSCHRG_DT'] = pd.to_datetime(df['NCH_BENE_DSCHRG_DT'].astype(str), format='%Y%m%d')
df['CLM_ADMSN_DT'] = pd.to_datetime(df['CLM_ADMSN_DT'].astype(str), format='%Y%m%d')

### Null Checks

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1332822 entries, 0 to 1332821
Data columns (total 81 columns):
 #   Column                          Non-Null Count    Dtype         
---  ------                          --------------    -----         
 0   DESYNPUF_ID                     1332822 non-null  object        
 1   CLM_ID                          1332822 non-null  int64         
 2   SEGMENT                         1332822 non-null  int64         
 3   CLM_FROM_DT                     1331533 non-null  float64       
 4   CLM_THRU_DT                     1331533 non-null  float64       
 5   PRVDR_NUM                       1332822 non-null  object        
 6   CLM_PMT_AMT                     1332822 non-null  float64       
 7   NCH_PRMRY_PYR_CLM_PD_AMT        1332822 non-null  float64       
 8   AT_PHYSN_NPI                    1319470 non-null  float64       
 9   OP_PHYSN_NPI                    776629 non-null   float64       
 10  OT_PHYSN_NPI                    154635 non

In [14]:
null_counts = df.isna().sum()
print(null_counts[null_counts > 0])

CLM_FROM_DT        1289
CLM_THRU_DT        1289
AT_PHYSN_NPI      13352
OP_PHYSN_NPI     556193
OT_PHYSN_NPI    1178187
                 ...   
HCPCS_CD_41     1332822
HCPCS_CD_42     1332822
HCPCS_CD_43     1332822
HCPCS_CD_44     1332822
HCPCS_CD_45     1332822
Length: 69, dtype: int64


In [15]:
## Drop nulls in ICD9_DGNS_CD_1
df = df.dropna(subset=['ICD9_DGNS_CD_1'])

### Remove Irrelevant Columns

#### Notes

* HCPCS_CD_1-HCPCS_CD_45: All of them are null for each value
* ADMTNG_ICD9_DGNS_CD: This is the reason they came in (For ex, shortness of breath). This is not something that matters long term in what we are looking at as we worry more about the condition reason for the stay documented in the ICD9_DGNS_CD_1-10 columns

In [16]:
df = df.drop(columns=['HCPCS_CD_1', 'HCPCS_CD_2','HCPCS_CD_3', 'HCPCS_CD_4', 'HCPCS_CD_5', 'HCPCS_CD_6', 'HCPCS_CD_7',
                      'HCPCS_CD_8', 'HCPCS_CD_9', 'HCPCS_CD_10', 'HCPCS_CD_11', 'HCPCS_CD_12','HCPCS_CD_13', 'HCPCS_CD_14',
                      'HCPCS_CD_15', 'HCPCS_CD_16','HCPCS_CD_17', 'HCPCS_CD_18', 'HCPCS_CD_19', 'HCPCS_CD_20','HCPCS_CD_21', 
                      'HCPCS_CD_22', 'HCPCS_CD_23', 'HCPCS_CD_24','HCPCS_CD_25', 'HCPCS_CD_26', 'HCPCS_CD_27', 'HCPCS_CD_28',
                      'HCPCS_CD_29', 'HCPCS_CD_30', 'HCPCS_CD_31', 'HCPCS_CD_32','HCPCS_CD_33', 'HCPCS_CD_34', 'HCPCS_CD_35', 
                      'HCPCS_CD_36','HCPCS_CD_37', 'HCPCS_CD_38', 'HCPCS_CD_39', 'HCPCS_CD_40','HCPCS_CD_41', 'HCPCS_CD_42', 
                      'HCPCS_CD_43', 'HCPCS_CD_44','HCPCS_CD_45','ADMTNG_ICD9_DGNS_CD'])

## Drop records where there isn't multiple stays for a patient

In [17]:
counts = df['DESYNPUF_ID'].value_counts()
duplicates = counts[counts > 1].index

df_duplicates = df[df['DESYNPUF_ID'].isin(duplicates)]

In [359]:
df_duplicates

,DESYNPUF_ID,CLM_ID,SEGMENT,CLM_FROM_DT,CLM_THRU_DT,PRVDR_NUM,CLM_PMT_AMT,NCH_PRMRY_PYR_CLM_PD_AMT,AT_PHYSN_NPI,OP_PHYSN_NPI,...,ICD9_DGNS_CD_7,ICD9_DGNS_CD_8,ICD9_DGNS_CD_9,ICD9_DGNS_CD_10,ICD9_PRCDR_CD_1,ICD9_PRCDR_CD_2,ICD9_PRCDR_CD_3,ICD9_PRCDR_CD_4,ICD9_PRCDR_CD_5,ICD9_PRCDR_CD_6
1,00016F745862898F,196201177000368,1,20090412.0,20090418.0,3900MB,26000.0,0.0,6.476809e+09,NaN,...,2724,19889,5849,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,00016F745862898F,196661177015632,1,20090831.0,20090902.0,3900HM,5000.0,0.0,6.119985e+08,6.119985e+08,...,NaN,NaN,NaN,NaN,7092.0,6186,V5866,NaN,NaN,NaN
3,00016F745862898F,196091176981058,1,20090917.0,20090920.0,3913XU,5000.0,0.0,4.971603e+09,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,00016F745862898F,196261176983265,1,20100626.0,20100701.0,3900MB,16000.0,0.0,6.408400e+09,1.960860e+09,...,7840,7856,4271,NaN,NaN,E8889,NaN,NaN,NaN,NaN
6,0007F12A492FD25D,196661176963773,1,20080919.0,20080922.0,3400WD,5000.0,0.0,8.956736e+09,6.551008e+09,...,78729,V103,34290,NaN,8659.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1332817,FFFE4E347796686D,992891161660972,1,20080910.0,20080929.0,5000NR,22000.0,0.0,3.058559e+09,4.837333e+09,...,4870,99859,4210,NaN,9904.0,30000,486,NaN,NaN,NaN
1332818,FFFE4E347796686D,992021161651979,1,20081125.0,20081202.0,5000NR,4000.0,0.0,3.058559e+09,NaN,...,34292,3970,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1332819,FFFE4E347796686D,992581161650328,1,20100201.0,20100204.0,5000NR,10000.0,0.0,1.665921e+09,NaN,...,42831,V4588,V4511,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1332820,FFFFEE90E1B4D0DF,992731161641904,1,20080220.0,20080224.0,1100CJ,6000.0,0.0,8.380401e+09,8.380401e+09,...,NaN,NaN,NaN,NaN,5534.0,NaN,NaN,NaN,NaN,NaN


### For now to make this process a little easier, replace the ICD code with descriptions

In [18]:
columns_to_replace = ['ICD9_DGNS_CD_1', 'ICD9_DGNS_CD_2', 'ICD9_DGNS_CD_3', 'ICD9_DGNS_CD_4',
                      'ICD9_DGNS_CD_5', 'ICD9_DGNS_CD_6', 'ICD9_DGNS_CD_7', 'ICD9_DGNS_CD_8',
                      'ICD9_DGNS_CD_9', 'ICD9_DGNS_CD_10']

# Replace ICD9 codes with long descriptions for each column in the list
for column in columns_to_replace:
    df_duplicates.loc[:,column + "_DESC"] = df_duplicates.loc[:,column].map(icd9_diag_dict).fillna(df_duplicates[column])

df_duplicates[["ICD9_DGNS_CD_1","ICD9_DGNS_CD_1_DESC"]]

,ICD9_DGNS_CD_1,ICD9_DGNS_CD_1_DESC
1,1970,Secondary malignant neoplasm of lung
2,6186,"Vaginal enterocele, congenital or acquired"
3,29623,"Major depressive affective disorder, single ep..."
4,3569,Unspecified hereditary and idiopathic peripher...
6,33811,Acute pain due to trauma
...,...,...
1332817,51881,Acute respiratory failure
1332818,43401,Cerebral thrombosis with cerebral infarction
1332819,41071,"Subendocardial infarction, initial episode of ..."
1332820,1890,"Malignant neoplasm of kidney, except pelvis"


In [19]:
# Still getting the setting with copy warning but we good. 
df_duplicates.loc[:,'CLM_DRG_CD_Description'] = df_duplicates.loc[:,'CLM_DRG_CD'].map(ms_drgs_dict).fillna(df_duplicates['CLM_DRG_CD'])

df_duplicates

,DESYNPUF_ID,CLM_ID,SEGMENT,CLM_FROM_DT,CLM_THRU_DT,PRVDR_NUM,CLM_PMT_AMT,NCH_PRMRY_PYR_CLM_PD_AMT,AT_PHYSN_NPI,OP_PHYSN_NPI,...,ICD9_DGNS_CD_2_DESC,ICD9_DGNS_CD_3_DESC,ICD9_DGNS_CD_4_DESC,ICD9_DGNS_CD_5_DESC,ICD9_DGNS_CD_6_DESC,ICD9_DGNS_CD_7_DESC,ICD9_DGNS_CD_8_DESC,ICD9_DGNS_CD_9_DESC,ICD9_DGNS_CD_10_DESC,CLM_DRG_CD_Description
1,00016F745862898F,196201177000368,1,20090412.0,20090418.0,3900MB,26000.0,0.0,6.476809e+09,NaN,...,Unspecified essential hypertension,"Chronic kidney disease, Stage III (moderate)",Aphasia,Hypopotassemia,"Osteoarthrosis, unspecified whether generalize...",Other and unspecified hyperlipidemia,Secondary malignant neoplasm of other specifie...,"Acute kidney failure, unspecified",NaN,Pneumothorax without CC/MCC
2,00016F745862898F,196661177015632,1,20090831.0,20090902.0,3900HM,5000.0,0.0,6.119985e+08,6.119985e+08,...,Other persistent mental disorders due to condi...,"Constipation, unspecified",NaN,NaN,NaN,NaN,NaN,NaN,NaN,Other Female Reproductive System O.R. Procedur...
3,00016F745862898F,196091176981058,1,20090917.0,20090920.0,3913XU,5000.0,0.0,4.971603e+09,NaN,...,"Other and unspecified alcohol dependence, unsp...","Arthropathy, unspecified, site unspecified","Epilepsy, unspecified, without mention of intr...",Personal history of noncompliance with medical...,Obstructive sleep apnea (adult)(pediatric),NaN,NaN,NaN,NaN,Disorders of Personality and Impulse Control
4,00016F745862898F,196261176983265,1,20100626.0,20100701.0,3900MB,16000.0,0.0,6.408400e+09,1.960860e+09,...,Unspecified essential hypertension,Lesion of ulnar nerve,Acquired absence of both cervix and uterus,"Retention of urine, unspecified",Unspecified protein-calorie malnutrition,Headache,Enlargement of lymph nodes,Paroxysmal ventricular tachycardia,NaN,Extensive O.R. Procedures Unrelated to Princip...
6,0007F12A492FD25D,196661176963773,1,20080919.0,20080922.0,3400WD,5000.0,0.0,8.956736e+09,6.551008e+09,...,"Unspecified gastritis and gastroduodenitis, wi...","Systolic heart failure, unspecified","Chronic airway obstruction, not elsewhere clas...",Personal history of other diseases of circulat...,Atrial fibrillation,Other dysphagia,Personal history of malignant neoplasm of breast,"Hemiplegia, unspecified, affecting unspecified...",NaN,Other Factors Influencing Health Status
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1332817,FFFE4E347796686D,992891161660972,1,20080910.0,20080929.0,5000NR,22000.0,0.0,3.058559e+09,4.837333e+09,...,Unspecified septicemia,"Acute kidney failure, unspecified","Shock, unspecified",Pulmonary collapse,Streptococcus infection in conditions classifi...,Influenza with pneumonia,Other postoperative infection,Acute and subacute bacterial endocarditis,NaN,Pneumothorax with MCC
1332818,FFFE4E347796686D,992021161651979,1,20081125.0,20081202.0,5000NR,4000.0,0.0,3.058559e+09,NaN,...,Benign neoplasm of colon,"Hemiplegia, unspecified, affecting unspecified...","Macular degeneration (senile), unspecified",Coronary atherosclerosis of native coronary ar...,"Diabetes with neurological manifestations, typ...","Hemiplegia, unspecified, affecting nondominant...",Diseases of tricuspid valve,NaN,NaN,031
1332819,FFFE4E347796686D,992581161650328,1,20100201.0,20100204.0,5000NR,10000.0,0.0,1.665921e+09,NaN,...,Need for prophylactic vaccination and inoculat...,Hyposmolality and/or hyponatremia,"Chronic kidney disease, unspecified",Other emphysema,"Congestive heart failure, unspecified",Acute diastolic heart failure,Status post administration of tPA (rtPA) in a ...,Renal dialysis status,NaN,Permanent Cardiac Pacemaker Implant with MCC
1332820,FFFFEE90E1B4D0DF,992731161641904,1,20080220.0,20080224.0,1100CJ,6000.0,0.0,8.380401e+09,8.380401e+09,...,"Osteoarthrosis, unspecified whether generalize...",Unspecified essential hypertension,NaN,NaN,NaN,NaN,NaN,NaN,NaN,692


In [20]:
df_duplicates["ICD9_PRCDR_CD_1"]

1             NaN
2          7092.0
3             NaN
4             NaN
6          8659.0
            ...  
1332817    9904.0
1332818       NaN
1332819       NaN
1332820    5534.0
1332821     393.0
Name: ICD9_PRCDR_CD_1, Length: 877883, dtype: float64

In [21]:
columns_to_replace = ['ICD9_PRCDR_CD_1', 'ICD9_PRCDR_CD_2', 'ICD9_PRCDR_CD_3', 
                      'ICD9_PRCDR_CD_4', 'ICD9_PRCDR_CD_5', 'ICD9_PRCDR_CD_6']

# Replace ICD9 codes with long descriptions for each column in the list
for column in columns_to_replace:
    df_duplicates.loc[:,column+"_DESC"] = df_duplicates[column].map(icd9_proc_dict).fillna(df_duplicates[column])

df_duplicates[["ICD9_PRCDR_CD_1","ICD9_PRCDR_CD_1_DESC"]]

,ICD9_PRCDR_CD_1,ICD9_PRCDR_CD_1_DESC
1,NaN,NaN
2,7092.0,Other operations on cul-de-sac
3,NaN,NaN
4,NaN,NaN
6,8659.0,Closure of skin and subcutaneous tissue of oth...
...,...,...
1332817,9904.0,Transfusion of packed cells
1332818,NaN,NaN
1332819,NaN,NaN
1332820,5534.0,Laparoscopic ablation of renal lesion or tissue


In [22]:
print(df_duplicates["ICD9_DGNS_CD_1_DESC"].value_counts())

ICD9_DGNS_CD_1_DESC
Pneumonia, organism unspecified                                                                   33556
Care involving other specified rehabilitation procedure                                           23573
Unspecified septicemia                                                                            22970
Obstructive chronic bronchitis with (acute) exacerbation                                          22784
Congestive heart failure, unspecified                                                             21611
                                                                                                  ...  
Late effects of viral encephalitis                                                                    1
Transverse or oblique presentation, delivered, with or without mention of antepartum condition        1
Unspecified viral hepatitis with hepatic coma                                                         1
Edema of penis                              

## Filter out what would be considered a readmission where penalty may be considered by Medicare

### Exclusions:

Based on Medicare definition of Readmissions, I think anything that isn't at an acute care facility isn't within the definition. 


- Admitted to PPS-exempt cancer hospital -> The column related (PRVDR_NUM) to this is completely random :) so no help

- W/O atleast 30-days of post-discharge enrollment in Medicare FFS (Will need the beneficiary sets for this)
- Discharged against medical advice (AMA)
    Discharge code 8 in Ful MedPAR-> ICD 9 doesn't have one so will use MS-DRGS to filter some out
- Admitted for Primary Psychiatic diagnosis (ICD_9_DGNS_1)
    - ICD-9 Codes 
- Admitted for Rehabilitation (CCS 254)
    - ICD-9 Codes: 'V520', 'V521', 'V524', 'V528', 'V529', 'V538', 'V570', 'V571', 'V572', 'V5721', 'V5722', 'V573', 'V574' 'V5781', 'V5789' 'V579', 'V5882'
- Admitted for Medical Treatment of Cancer (CCS 45)
    - ICD-9 Codes: 'V580', 'V581', 'V5811', 'V5812', 'V661', 'V662', 'V671', 'V672'   
- General always planned procedures
    - Bone Marrow Proc (CCS 64) = '410', '4100', '4101', '4102', '4103', '4104', '4105', '4106', '4107', '4108', '4109'
    - Kidney Transplant (CCS 105) = '5561', '5569' 
    - Other Organ Transplantation (176) = '335','3350', '3351', '3352', '336', '375', '3751', '4194', '4697', '5051', '5059', '5280', '5281', '5282', '5283', '5284', '5285', '5286'

[Reference on Medicare Measures by CMS](https://qualitynet.cms.gov/inpatient/measures/readmission/methodology#:~:text=2024%20All%2DCause%20Hospital%2DWide%20Measure%20Updates%20and%20Specifications%20Report%3A%20Hospital%2DWide%20Readmission)

[Reference for Planned Readmissions ICD-9 Related Codes](https://www.cms.gov/medicare/quality-initiatives-patient-assessment-instruments/qualitymeasures/downloads/fy-2017_icd10-and-quality-measures_faqs.pdf)
- [Diagnoses](https://hcup-us.ahrq.gov/toolssoftware/ccs/AppendixASingleDX.txt)
- [Procedures](https://hcup-us.ahrq.gov/toolssoftware/ccs/AppendixBSinglePR.txt)

### Create Risk Variables

In [ ]:
risk_maps = {
    'Severe Infection': [1,3,4,5,6],
    'Sepsis/Shock': [2],
    'Infectious Disease/Pneumonias' : [7,114,115,116],
    'Diabetes and/or Complications' : [17,18,19,122,123],
    'Protein-Calorie Malnutrition' : [21],
    'Endocrine/Metabolic/Fluid/Electrolyte Disorders' : [23,24],
    'End Stage Liver Disease/Cirrhosis' : [27,28],
    'Gastrointestinal Disorders' : [34,36],
    'Rheumatoid Arthritis and Inflammatory Connective Tissue Disease' : [40],
    'Severe Hematological Disorders': [46],
    'Coagulation Defects and Other Hematological Disorders': [48],
    'Iron deficiency or other/unspecified anemias and blood disease': [49],
    'Drug/alcohol psychosis or dependence': [54, 55],
    'Psychiatric comorbidity': [57,58,59,61,63],
    'Hemiplegia, paraplegia, paralysis, functional disability': [70,71,72,73,74,103,104,189,190],
    'Seizure Disorders And Convulsions': [79],
    'Respirator dependence/tracheostomy status': [82], 
    'Cardio-respiratory failure and shock': [84], #plus ICD-10-CM codes R09.01 and R09.02 TODO: Figure out the 9 Mapping for this
    'Congestive heart failure': [85],
    'Coronary atherosclerosis or angina, cerebrovascular disease': [86,87,88,89,102,105,106,107,108,109],
    'Specified arrhythmias and other heart rhythm disorders': [96,97],
    'Chronic obstructive pulmonary disease (COPD)': [111],
    'Fibrosis of lung or other chronic lung disorders': [112],
    'Transplants': [132,186],
    'Dialysis status': [134],
    'Renal failure': [135, 136,137,138,139,140],
    'Decubitus ulcer or chronic skin ulcer': [157,158,159,160,161],
    'Hip fracture/dislocation': [170]
}

In [ ]:
#TODO: Fix this as this was the testing one df_duplicates["Severe Infection"] = df['ICD9_DGNS_CD_1'].isin(risk_maps["Severe Infection"]).astype(int)

In [ ]:

#TODO: Make df_duplicates['Years over 65'] Risk factor

### Remove stays relevant to rehabilitation

In [365]:
rehab = ['V520', 'V521', 'V524', 'V528', 'V529', 'V538', 'V570', 'V571', 'V572', 'V5721', 'V5722', 'V573', 
         'V574' 'V5781', 'V5789' 'V579', 'V5882']

before_count = df_duplicates.shape[0]
df_duplicates= df_duplicates[~df_duplicates['ICD9_DGNS_CD_1'].isin(rehab)]
after_count = df_duplicates.shape[0]

print("Before: ",before_count,  "\nAfter: ", after_count)

Before:  877883 
After:  877402


### Remove those recieving cancer treatment

In [366]:
cancer_treatments = ['V580', 'V581', 'V5811', 'V5812', 'V661', 'V662', 'V671', 'V672']

before_count = df_duplicates.shape[0]

# Find patients who have at least one row with an ICD-9 code in the list
cancer_patients = df_duplicates[df_duplicates['ICD9_DGNS_CD_1'].isin(cancer_treatments)]['DESYNPUF_ID'].unique()

df_duplicates= df_duplicates[~df_duplicates['DESYNPUF_ID'].isin(cancer_patients)]

after_count = df_duplicates.shape[0]
print("Before: ",before_count,  "\nAfter: ", after_count)

Before:  877402 
After:  869673


### Remove those with Cancer in Gen? Confused if its just those recieving treatment or naw

In [367]:
cancer = ['1400', '1401', '1403', '1404', '1405', '1406', '1408', '1409', '1410', '1411', '1412', '1413', '1414', 
             '1415', '1416', '1418', '1419', '1420', '1421', '1422', '1428', '1429', '1430', '1431', '1438', '1439', 
             '1440', '1441', '1448', '1449', '1450', '1451', '1452', '1453', '1454', '1455', '1456', '1458', '1459', 
             '1460', '1461', '1462', '1463', '1464', '1465', '1466', '1467', '1468', '1469', '1470', '1471', '1472', 
             '1473', '1478', '1479', '1480', '1481', '1482', '1483', '1488', '1489', '1490', '1491', '1498', '1499', 
             '1600', '1601', '1602', '1603', '1604', '1605', '1608', '1609', '1610', '1611', '1612', '1613', '1618', 
             '1619', '1950', '2300', '2310', 'V1001', 'V1002', 'V1021', '1500', '1501', '1502', '1503', '1504', 
             '1505', '1508', '1509', '2301', 'V1003', '1510', '1511', '1512', '1513', '1514', '1515', '1516', '1518', 
             '1519', '20923', '2302', 'V1004', '1530', '1531', '1532', '1533', '1534', '1535', '1536', '1537', '1538', 
             '1539', '1590', '20910', '20911', '20912', '20913', '20914', '20915', '20916', '2303', 'V1005', '1540', 
             '1541', '1542', '1543', '1548', '20917', '2304', '2305', '2306', '79670', '79671', '79672', '79673', '79676',
             'V1006', '1550', '1551', '1552', '2308', 'V1007', '1570', '1571', '1572', '1573', '1574', '1578', '1579', 
             '1520', '1521', '1522', '1523', '1528', '1529', '1560', '1561', '1562', '1568', '1569', '1580', '1588', 
             '1589', '1591', '1598', '1599', '20900', '20901', '20902', '20903', '2307', '2309', 'V1000', 'V1009', 
             '1622', '1623', '1624', '1625', '1628', '1629', '20921', '2312', 'V1011', '1620', '1630', '1631', '1638',
             '1639', '1650', '1658', '1659', '2311', '2318', '2319', 'V101', 'V1020', 'V1022', '1700', '1701', '1702', 
             '1703', '1704', '1705', '1706', '1707', '1708', '1709', '1710', '1712', '1713', '1714', '1715', '1716', '1717', 
             '1718', '1719', '1740', '1741', '1742', '1743', '1744', '1745', '1746', '1748', '1749', '1750', '1759',
             '2330', 'V103', '179', '1820', '1821', '1828', '2332', 'V1042', '1800', '1801', '1808', '1809', '2331', 
             '7950', '79501', '79502', '79503', '79504', '79506', 'V1041', '1830', 'V1043', '181', '1832', '1833', 
             '1834', '1835', '1838', '1839', '1840', '1841', '1842', '1843', '1844', '1848', '1849', '2333', 
             '23330', '23331', '23332', '23339', '79516', 'V1040', 'V1044', '185', '2334', 'V1046', '1860', '1869', 
             'V1047', '1871', '1872', '1873', '1874', '1875', '1876', '1877', '1878', '1879', '2335', '2336', 'V1045', 
             'V1048', 'V1049', '1880', '1881', '1882', '1883', '1884', '1885', '1886', '1887', '1888', '1889', '2337', 'V1051', 
             '1890', '1891', '20924', 'V1052', 'V1053', '1892', '1893', '1894', '1898', '1899', '2339', 'V1050', 'V1059', 
             '1910', '1911', '1912', '1913', '1914', '1915', '1916', '1917', '1918', '1919', '1920', '1921', '1922', '1923', 
             '1928', '1929', 'V1085', 'V1086', '193', '25802', '25803', 'V1087']


before_count = df_duplicates.shape[0]

# Find patients who have at least one row with an ICD-9 code in the list
cancer_patients = df_duplicates[df_duplicates['ICD9_DGNS_CD_1'].isin(cancer)]['DESYNPUF_ID'].unique()

df_duplicates= df_duplicates[~df_duplicates['DESYNPUF_ID'].isin(cancer)]

after_count = df_duplicates.shape[0]
print("Before: ",before_count,  "\nAfter: ", after_count)

Before:  869673 
After:  869673


### Remove those discharged AMA

In [368]:
before_count = df_duplicates.shape[0]
df_duplicates = df_duplicates[~df_duplicates['CLM_DRG_CD_Description'].str.contains('AMA', case=False, na=False)]
after_count = df_duplicates.shape[0]

print("Before: ",before_count,  "\nAfter: ", after_count)

Before:  869673 
After:  868293


### Remove stays relevant to psychriatric disorders

In [369]:
psych = ['29381', '29382', '29500', '29501', '29502', '29503', '29504', '29505', '29510', '29511', '29512', 
         '29513', '29514', '29515', '29520', '29521', '29522', '29523', '29524', '29525', '29530', '29531', 
         '29532', '29533', '29534', '29535', '29540', '29541', '29542', '29543', '29544', '29545', '29550', 
         '29551', '29552', '29553', '29554', '29555', '29560', '29561', '29562', '29563', '29564', '29565', 
         '29570', '29571', '29572', '29573', '29574', '29575', '29580', '29581', '29582', '29583', '29584', 
         '29585', '29590', '29591', '29592', '29593', '29594', '29595', '2970', '2971', '2972', '2973', 
         '2978', '2979', '2980', '2981', '2982', '2983', '2984','2988', '2989']

before_count = df_duplicates.shape[0]
df_duplicates= df_duplicates[~df_duplicates['ICD9_DGNS_CD_1'].isin(psych)]
after_count = df_duplicates.shape[0]

print("Before: ",before_count,  "\nAfter: ", after_count)

Before:  868293 
After:  855651


## Subset with just readmissions, any cause

In [370]:
# Sort by Beneficiary ID and Admission Date to make math easier
df_sorted = df_duplicates.sort_values(by=['DESYNPUF_ID', 'CLM_ADMSN_DT'])  

# Calculate the time difference between admissions
df_sorted['ADMISSION_DIFF'] = df_sorted.groupby('DESYNPUF_ID')['CLM_ADMSN_DT'].diff().dt.days

# Calculate the time difference between discharge and the next admission
df_sorted['DISCHARGE_TO_NEXT_ADMISSION'] = (df_sorted.groupby('DESYNPUF_ID')['CLM_ADMSN_DT'].shift(-1) - df_sorted['NCH_BENE_DSCHRG_DT']).dt.days

df_sorted[['DESYNPUF_ID','CLM_ADMSN_DT','ADMISSION_DIFF','ICD9_DGNS_CD_1','ICD9_DGNS_CD_2','CLM_DRG_CD_Description','CLM_DRG_CD']]


,DESYNPUF_ID,CLM_ADMSN_DT,ADMISSION_DIFF,ICD9_DGNS_CD_1,ICD9_DGNS_CD_2,CLM_DRG_CD_Description,CLM_DRG_CD
733454,00000B48BCF4AD29,2008-07-07,NaN,4241,2760,Acute Myocardial Infarction Discharged Alive w...,281
733455,00000B48BCF4AD29,2008-07-24,17.0,42823,2859,Cardiac Arrest Unexplained with MCC,296
733456,00000B48BCF4AD29,2008-08-05,12.0,3970,2800,Vein Ligation and Stripping,263
733457,0000525AB30E4DEF,2008-05-25,NaN,5609,42732,Anal and Stomal Procedures without CC/MCC,349
733458,0000525AB30E4DEF,2008-09-18,116.0,5579,V4611,Gastrointestinal Obstruction with MCC,388
...,...,...,...,...,...,...,...
733451,FFFFBC57C82B76C5,2008-11-06,NaN,41071,4280,Acute and Subacute Endocarditis with CC,289
733452,FFFFBC57C82B76C5,2008-11-12,6.0,99811,4019,Wound Debridements for Injuries with MCC,901
733453,FFFFBC57C82B76C5,2008-11-22,10.0,41401,4111,238,238
1332820,FFFFEE90E1B4D0DF,2008-02-20,NaN,1890,71590,692,692


In [371]:
df_sorted[['DESYNPUF_ID','CLM_ADMSN_DT','NCH_BENE_DSCHRG_DT','DISCHARGE_TO_NEXT_ADMISSION','ICD9_DGNS_CD_1_DESC','CLM_DRG_CD_Description']]

,DESYNPUF_ID,CLM_ADMSN_DT,NCH_BENE_DSCHRG_DT,DISCHARGE_TO_NEXT_ADMISSION,ICD9_DGNS_CD_1_DESC,CLM_DRG_CD_Description
733454,00000B48BCF4AD29,2008-07-07,2008-07-18,6.0,Aortic valve disorders,Acute Myocardial Infarction Discharged Alive w...
733455,00000B48BCF4AD29,2008-07-24,2008-07-26,10.0,Acute on chronic systolic heart failure,Cardiac Arrest Unexplained with MCC
733456,00000B48BCF4AD29,2008-08-05,2008-08-20,NaN,Diseases of tricuspid valve,Vein Ligation and Stripping
733457,0000525AB30E4DEF,2008-05-25,2008-06-05,105.0,Unspecified intestinal obstruction,Anal and Stomal Procedures without CC/MCC
733458,0000525AB30E4DEF,2008-09-18,2008-09-24,NaN,Unspecified vascular insufficiency of intestine,Gastrointestinal Obstruction with MCC
...,...,...,...,...,...,...
733451,FFFFBC57C82B76C5,2008-11-06,2008-11-11,1.0,"Subendocardial infarction, initial episode of ...",Acute and Subacute Endocarditis with CC
733452,FFFFBC57C82B76C5,2008-11-12,2008-11-14,8.0,Hemorrhage complicating a procedure,Wound Debridements for Injuries with MCC
733453,FFFFBC57C82B76C5,2008-11-22,2008-11-24,NaN,Coronary atherosclerosis of native coronary ar...,238
1332820,FFFFEE90E1B4D0DF,2008-02-20,2008-02-24,75.0,"Malignant neoplasm of kidney, except pelvis",692


In [372]:
# For any-cause readmissions, check if the admission is within 30 days of the previous one as we don't realy care about ICD code
df_sorted['READMISSION'] = df_sorted['DISCHARGE_TO_NEXT_ADMISSION'] <= 30

df_sorted[['DESYNPUF_ID','CLM_ADMSN_DT','NCH_BENE_DSCHRG_DT','READMISSION','DISCHARGE_TO_NEXT_ADMISSION','ICD9_DGNS_CD_1','ICD9_DGNS_CD_2','CLM_DRG_CD_Description','CLM_DRG_CD']]

,DESYNPUF_ID,CLM_ADMSN_DT,NCH_BENE_DSCHRG_DT,READMISSION,DISCHARGE_TO_NEXT_ADMISSION,ICD9_DGNS_CD_1,ICD9_DGNS_CD_2,CLM_DRG_CD_Description,CLM_DRG_CD
733454,00000B48BCF4AD29,2008-07-07,2008-07-18,True,6.0,4241,2760,Acute Myocardial Infarction Discharged Alive w...,281
733455,00000B48BCF4AD29,2008-07-24,2008-07-26,True,10.0,42823,2859,Cardiac Arrest Unexplained with MCC,296
733456,00000B48BCF4AD29,2008-08-05,2008-08-20,False,NaN,3970,2800,Vein Ligation and Stripping,263
733457,0000525AB30E4DEF,2008-05-25,2008-06-05,False,105.0,5609,42732,Anal and Stomal Procedures without CC/MCC,349
733458,0000525AB30E4DEF,2008-09-18,2008-09-24,False,NaN,5579,V4611,Gastrointestinal Obstruction with MCC,388
...,...,...,...,...,...,...,...,...,...
733451,FFFFBC57C82B76C5,2008-11-06,2008-11-11,True,1.0,41071,4280,Acute and Subacute Endocarditis with CC,289
733452,FFFFBC57C82B76C5,2008-11-12,2008-11-14,True,8.0,99811,4019,Wound Debridements for Injuries with MCC,901
733453,FFFFBC57C82B76C5,2008-11-22,2008-11-24,False,NaN,41401,4111,238,238
1332820,FFFFEE90E1B4D0DF,2008-02-20,2008-02-24,False,75.0,1890,71590,692,692


Need to mark the index stays as they are recorded as false so that way we can remove everything else. 10/10 Probably a simpiler way but this works in the end

In [373]:
df_sorted['RELATED'] = df_sorted.groupby('DESYNPUF_ID')['READMISSION'].shift(1, fill_value=False)

# The last stay in a readmission chain should be flagged as part of the chain if it follows a readmission
df_sorted['RELATED'] = df_sorted['READMISSION'] | df_sorted['RELATED']

all_cause = df_sorted[df_sorted['RELATED']]

In [375]:
all_cause[['DESYNPUF_ID','CLM_ADMSN_DT','NCH_BENE_DSCHRG_DT','READMISSION','DISCHARGE_TO_NEXT_ADMISSION','ICD9_DGNS_CD_1_DESC','ICD9_PRCDR_CD_1_DESC']]

,DESYNPUF_ID,CLM_ADMSN_DT,NCH_BENE_DSCHRG_DT,READMISSION,DISCHARGE_TO_NEXT_ADMISSION,ICD9_DGNS_CD_1_DESC,ICD9_PRCDR_CD_1_DESC
733454,00000B48BCF4AD29,2008-07-07,2008-07-18,True,6.0,Aortic valve disorders,Replacement of aortic valve with tissue graft
733455,00000B48BCF4AD29,2008-07-24,2008-07-26,True,10.0,Acute on chronic systolic heart failure,NaN
733456,00000B48BCF4AD29,2008-08-05,2008-08-20,False,NaN,Diseases of tricuspid valve,Single internal mammary-coronary artery bypass
999387,0000838E2BBC2ADA,2008-04-17,2008-05-05,True,7.0,Care involving other specified rehabilitation ...,NaN
999388,0000838E2BBC2ADA,2008-05-12,2008-05-22,False,NaN,Obstructive chronic bronchitis with (acute) ex...,Other endoscopy of small intestine
...,...,...,...,...,...,...,...
999384,FFFEBB75660834C6,2008-02-10,2008-02-19,True,29.0,Pneumonitis due to inhalation of food or vomitus,Non-invasive mechanical ventilation
999385,FFFEBB75660834C6,2008-03-19,2008-03-27,False,NaN,Aortic valve disorders,Other replacement of aortic valve
733451,FFFFBC57C82B76C5,2008-11-06,2008-11-11,True,1.0,"Subendocardial infarction, initial episode of ...","Venous catheterization, not elsewhere classified"
733452,FFFFBC57C82B76C5,2008-11-12,2008-11-14,True,8.0,Hemorrhage complicating a procedure,Excision of lesion of other soft tissue


## Readmission with Same Principal Diagnosis

In [379]:
all_cause['SAME_CAUSE_READMISSION'] = (all_cause['ICD9_DGNS_CD_1'] == all_cause.groupby('DESYNPUF_ID')['ICD9_DGNS_CD_1'].shift(1))



C:\Users\nicol\AppData\Local\Temp\ipykernel_28164\536439387.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_cause['SAME_CAUSE_READMISSION'] = (all_cause['ICD9_DGNS_CD_1'] == all_cause.groupby('DESYNPUF_ID')['ICD9_DGNS_CD_1'].shift(1))


In [391]:
all_cause['RELATED'] = all_cause.groupby('DESYNPUF_ID')['SAME_CAUSE_READMISSION'].shift(1,fill_value=False)

C:\Users\nicol\AppData\Local\Temp\ipykernel_28164\4266168964.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_cause['RELATED'] = all_cause.groupby('DESYNPUF_ID')['SAME_CAUSE_READMISSION'].shift(1,fill_value=False)


C:\Users\nicol\AppData\Local\Temp\ipykernel_28164\3593791568.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_cause['RELATED'] = all_cause['SAME_CAUSE_READMISSION'] | df_sorted['RELATED']


In [394]:
all_cause[['DESYNPUF_ID','CLM_ADMSN_DT','NCH_BENE_DSCHRG_DT','SAME_CAUSE_READMISSION','RELATED','DISCHARGE_TO_NEXT_ADMISSION','ICD9_DGNS_CD_1_DESC','ICD9_PRCDR_CD_1_DESC']]

,DESYNPUF_ID,CLM_ADMSN_DT,NCH_BENE_DSCHRG_DT,SAME_CAUSE_READMISSION,RELATED,DISCHARGE_TO_NEXT_ADMISSION,ICD9_DGNS_CD_1_DESC,ICD9_PRCDR_CD_1_DESC
733454,00000B48BCF4AD29,2008-07-07,2008-07-18,False,True,6.0,Aortic valve disorders,Replacement of aortic valve with tissue graft
733455,00000B48BCF4AD29,2008-07-24,2008-07-26,False,True,10.0,Acute on chronic systolic heart failure,NaN
733456,00000B48BCF4AD29,2008-08-05,2008-08-20,False,True,NaN,Diseases of tricuspid valve,Single internal mammary-coronary artery bypass
999387,0000838E2BBC2ADA,2008-04-17,2008-05-05,False,True,7.0,Care involving other specified rehabilitation ...,NaN
999388,0000838E2BBC2ADA,2008-05-12,2008-05-22,False,True,NaN,Obstructive chronic bronchitis with (acute) ex...,Other endoscopy of small intestine
...,...,...,...,...,...,...,...,...
999384,FFFEBB75660834C6,2008-02-10,2008-02-19,False,True,29.0,Pneumonitis due to inhalation of food or vomitus,Non-invasive mechanical ventilation
999385,FFFEBB75660834C6,2008-03-19,2008-03-27,False,True,NaN,Aortic valve disorders,Other replacement of aortic valve
733451,FFFFBC57C82B76C5,2008-11-06,2008-11-11,False,True,1.0,"Subendocardial infarction, initial episode of ...","Venous catheterization, not elsewhere classified"
733452,FFFFBC57C82B76C5,2008-11-12,2008-11-14,False,True,8.0,Hemorrhage complicating a procedure,Excision of lesion of other soft tissue


In [385]:


# The last stay in a readmission chain should be flagged as part of the chain if it follows a readmission
all_cause['RELATED'] = all_cause.groupby('DESYNPUF_ID')['SAME_CAUSE_READMISSION'].shift(1, fill_value=False)
all_cause['RELATED'] = all_cause['RELATED'].where(all_cause['DESYNPUF_ID'] == all_cause['DESYNPUF_ID'].shift(1), False)

C:\Users\nicol\AppData\Local\Temp\ipykernel_28164\1777221859.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_cause['RELATED'] = all_cause.groupby('DESYNPUF_ID')['SAME_CAUSE_READMISSION'].shift(1, fill_value=False)
C:\Users\nicol\AppData\Local\Temp\ipykernel_28164\1777221859.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_cause['RELATED'] = all_cause['RELATED'].where(all_cause['DESYNPUF_ID'] == all_cause['DESYNPUF_ID'].shift(1), False)


## Readmission with Same Principal and Secondary Diagnosis

In [318]:
all_cause['ICD9_PRCDR_CD_1_DESC'].value_counts()

ICD9_PRCDR_CD_1_DESC
Transfusion of packed cells                                             9115
Venous catheterization, not elsewhere classified                        7279
Hemodialysis                                                            6904
Total knee replacement                                                  6077
Excision of lingual thyroid                                             5964
                                                                        ... 
Insertion of breast tissue expander                                        1
Intubation of nasolacrimal duct                                            1
Injection of destructive agent into spinal canal                           1
Resection of vessel with anastomosis, other vessels of head and neck       1
Repair of blepharoptosis by other techniques                               1
Name: count, Length: 1795, dtype: int64

## Readmission with Same MS-DRG

In [313]:
values = all_cause['ICD9_DGNS_CD_1_DESC'].value_counts()

print(values)

ICD9_DGNS_CD_1_DESC
Pneumonia, organism unspecified                                                     11168
Care involving other specified rehabilitation procedure                              9527
Unspecified septicemia                                                               8623
Congestive heart failure, unspecified                                                7662
Obstructive chronic bronchitis with (acute) exacerbation                             7394
                                                                                    ...  
Progressive multifocal leukoencephalopathy                                              1
Benign neoplasm of uterus, part unspecified                                             1
Severe pre-eclampsia, delivered, with or without mention of antepartum condition        1
Tick-borne rickettsiosis, unspecified                                                   1
Neoplasm of uncertain behavior of meninges                                      